# Feature Extraction


In [ ]:
import os, sys

sys.path.append(os.path.abspath(os.path.join("../..")))

import pandas as pd
from src.feature_extraction.preprocess import (
    preprocess_text,
    plot_most_common_tokens,
    obfuscate_and_filter_tokens,
    plot_combined_percentage,
)
from src.feature_extraction.tf_idf_vectorization import vectorize, cross_validation
import joblib


dataset_path = "../../data/mixed_dataset.csv"
data = pd.read_csv(dataset_path, na_filter=False)

Tokenization will follow these steps:

1. Tokenization:
   Tokenization breaks text into individual words or tokens.

2. Lowercasing:
   Converting all text to lowercase to ensure uniformity.

3. Removing Stopwords:
   Removing common stopwords (e.g., 'and', 'the', 'is') that don't carry significant meaning.

4. Removing Punctuation:
   Removing punctuation marks that aren't crucial for detecting phishing emails.

5. Stemming or Lemmatization:
   Reducing words to their base or root form to handle different variations of words.


In [ ]:
plot_most_common_tokens(data)

As depicted in the plot, numerous unnecessary tokens, primarily consisting of punctuation marks and stopwords, are evident. Our next step involves their removal to enhance the data quality.


In [ ]:
# Applying preprocessing
data["Body"] = data["Body"].apply(preprocess_text)
data["Subject"] = data["Subject"].apply(preprocess_text)
data["From"] = data["From"].apply(preprocess_text)

plot_most_common_tokens(data)

After the initial tokenization process, which involved tasks like removing stopwords, punctuation, and applying lemmatization, distinct sets of frequent words became apparent in each dataset. Specifically, in the phishing dataset, the terms "monkeyorg" and "jose" are noteworthy due to their uniqueness, high frequency, and lack of informativeness. On the other hand, in the legitimate dataset, the term "enron" stands out among the frequently occurring words.

To address this concern, it becomes crucial to filter out these uninformative and frequently occurring terms before proceeding with machine training.

The plots provided below illustrate the tokens with the highest percentage for each dataset. Our goal in carefully examining these visual representations is to pinpoint and subsequently exclude tokens that either carry dataset-specific characteristics or lack informativeness. This step ensures that the machine learning model to be trained utilizes a more refined and discriminative feature set, improving its ability to generalize effectively and make meaningful distinctions between phishing and legitimate content.


In [ ]:
plot_combined_percentage(data)

In [ ]:
data["Body"] = data["Body"].apply(obfuscate_and_filter_tokens)
data["Subject"] = data["Subject"].apply(obfuscate_and_filter_tokens)
data["From"] = data["From"].apply(obfuscate_and_filter_tokens)

## Cross Validation for TF-IDF Parameters


In situations where there's a significant class imbalance, accuracy can be misleading. For our case, the legitimate class is much larger than the phishing class, a model that simply predicts the majority class (legitimate) for every instance could achieve a high accuracy. However, such a model would have low sensitivity (recall) for the minority class (phishing), leading to a low F1 score.

By using the F1 score as our evaluation metric, we prioritize a balance between precision and recall, making it more suitable for our imbalanced dataset and providing a more comprehensive assessment of our model's effectiveness in correctly identifying both phishing and legitimate instances.


In [ ]:
# ngram range hyper-parameter fine tuning for linear SVC
cross_validation(data)

In [ ]:
# Get TF-IDF features with ngram range (1, 2)
resulting_features, tfidf_vectorizer = vectorize(data)
print(resulting_features)

# Save TF-IDF features
joblib.dump(resulting_features, "tfidf_features.pkl")

## Domain Specific Features


In [ ]:
from src.feature_extraction.extract_features import extract_domain_features
from sklearn.preprocessing import MinMaxScaler

domain_specific_features = (data["Subject"] + data["Body"]).apply(
    extract_domain_features
)
domain_specific_features = pd.json_normalize(domain_specific_features)
features_to_scale = domain_specific_features.columns

scaler = MinMaxScaler()
scaled_features = scaler.fit_transform(domain_specific_features)
scaled_features_df = pd.DataFrame(scaled_features, columns=features_to_scale)

scaled_features_df.head()
scaled_features_df.to_csv("domain_specific_features.csv", index=False)